In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score

In [24]:
# Load dataset
data_dir = 'data_readinglevel'
x_train = pd.read_csv(os.path.join(data_dir, 'x_train.csv'))
y_train = pd.read_csv(os.path.join(data_dir, 'y_train.csv'))

texts = x_train['text'].tolist()
labels = (y_train['Coarse Label'] == 'Key Stage 4-5').astype(int)

vectorizer = CountVectorizer(
    lowercase=True,
    stop_words='english',
    token_pattern=r'(?u)\b\w*[^\d\W]\w*\b',
)

X = vectorizer.fit_transform(texts)
vectorizer.get_feature_names_out()

analyze = vectorizer.build_analyzer()
analyze("This is a text document to analyze 102.")





['text', 'document', 'analyze']